## Constructing img2gene Data

### Please Download row Data

### Constructing RNA and Image Data

In [29]:
import warnings
import anndata
warnings.filterwarnings("ignore", category=UserWarning, module='anndata')
warnings.filterwarnings("ignore", category=UserWarning, module="scanpy")
warnings.filterwarnings("ignore", category=anndata.ImplicitModificationWarning)

from importlib import reload
from VAE_utils import match_and_filter_adata_lists
import preprocess 
reload(preprocess)
from preprocess import AnnDataProcessor, Config
AnnData = AnnDataProcessor()
from PIL import Image
Image.MAX_IMAGE_PIXELS = 200000000

FALSE


##### Row image

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
file_dir = 'DownloadRowDataFile/'
# img_names = [151507, 151508, 151509, 151510, 151669,151670, 151671, 151672, 151673, 151674, 151675, 151676]

img_names = [151673, 151674, 151675, 151676]
xycoords_list = []
img_coordinates_list = []
img_list = []
barcodes_list = []
for i, sn in enumerate(img_names):
    data = pd.read_csv(f"{file_dir}/{sn}/spatial/tissue_positions_list.csv", header=None)
    img = np.array(Image.open(f"{file_dir}/{sn}/spatial/{sn}_full_image.tif"))
    x_y_coords = data.iloc[:, -4:-2].values[:,[1, 0]]  # Get the 4th and 3rd to last columns (x, y)
    last_two_columns = data.iloc[:, -2:].values  # Get the last two columns
    xycoords_list.append(x_y_coords)
    img_coordinates_list.append(last_two_columns)
    img_list.append(img)
    barcodes_list.append(data.iloc[:, -0].values)

In [19]:
adata_img_list = AnnData.create_img_adata_list_from_data(img_list, barcodes_list, img_coordinates_list, xycoords_list, patch_size = 32)

Vectorizing and cropping 32x32 image patches from 4992 coordinate points...
AnnData object construction complete!
Vectorizing and cropping 32x32 image patches from 4992 coordinate points...
AnnData object construction complete!
Vectorizing and cropping 32x32 image patches from 4992 coordinate points...
AnnData object construction complete!
Vectorizing and cropping 32x32 image patches from 4992 coordinate points...
AnnData object construction complete!


##### Row RNA

In [20]:
import scanpy as sc
import pandas as pd

def get_adata(sn, dir):
    # Set up input files and paths
    input_dir = f'{dir}/{sn}/'
    count_file = f'{input_dir}{sn}_filtered_feature_bc_matrix.h5'
    metadata_file = f'{input_dir}metadata.tsv'

    # 1. Load Visium data
    adata = sc.read_visium(path=input_dir, count_file=count_file)
    adata.obsm["spatial"] = adata.obs.loc[:, ['array_col', 'array_row']].to_numpy()

    # 2. Read metadata file
    metadata = pd.read_csv(metadata_file, sep='\t', index_col=0)

    # 3. Merge cell types into adata
    adata.obs = adata.obs.join(metadata['layer_guess_reordered'])
    adata.obs.rename(columns={'layer_guess_reordered': 'cell_type'}, inplace=True)

    # 4. Remove cells with 'NA' cell types
    print(f"Cells before filtering: {adata.n_obs}")
    adata = adata[adata.obs['cell_type'].notna() & (adata.obs['cell_type'] != 'NA')].copy()
    print(f"Cells after filtering NA cell types: {adata.n_obs}")
    sc.pp.filter_cells(adata, min_counts=10)
    # Remove genes expressed in fewer than 10 cells
    sc.pp.filter_genes(adata, min_cells=10)
    
    # 5. Preprocessing and highly variable gene selection
    # To keep adata.X as raw data, we create a copy for calculations
    del adata.uns
    adata.var_names_make_unique()
    return adata

adata_rna_list = []

# Assuming 'img_names' and 'file_dir' are defined elsewhere in your code
for sn in img_names:
    adata = get_adata(sn, file_dir)
    adata_rna_list.append(adata)

Cells before filtering: 3639
Cells after filtering NA cell types: 3611
Cells before filtering: 3673
Cells after filtering NA cell types: 3635
Cells before filtering: 3592
Cells after filtering NA cell types: 3566
Cells before filtering: 3460
Cells after filtering NA cell types: 3431


#### squidpy for data

In [21]:
import squidpy as sq
import scanpy as sc
import copy
from collections import Counter

print("\nFinding spatially variable genes (SVGs) present in at least 2 samples...")

# --- Parameters ---
p_val_threshold = 0.05
min_sample_occurrence = 2 # A gene must be an SVG in at least this many samples

# This list will store the set of SVG genes from each sample
svgs_per_sample = []
adata_rna_list_new = copy.deepcopy(adata_rna_list)

# --- 1. Loop through each sample to find SVGs ---
for i, adata in enumerate(adata_rna_list):
    sample_name = img_names[i] if 'img_names' in locals() else str(i)
    print(f"--- Processing sample: {sample_name} ---")

    # Preprocessing
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    # Compute spatial neighbors graph
    sq.gr.spatial_neighbors(adata)

    # Compute spatial autocorrelation (Moran's I)
    sq.gr.spatial_autocorr(adata, mode="moran")

    # Filter for significant SVGs for the current sample
    results_df = adata.uns['moranI']
    current_svgs = set(results_df[results_df['pval_norm_fdr_bh'] < p_val_threshold].index)
    
    print(f"Found {len(current_svgs)} significant SVGs in sample {sample_name}")
    svgs_per_sample.append(current_svgs)

# --- 2. Count gene occurrences and filter ---
# Create a single counter to store the frequency of each gene across all samples
gene_counts = Counter()
for gene_set in svgs_per_sample:
    gene_counts.update(gene_set)

# Filter the counter to get genes that appear in at least `min_sample_occurrence` samples
svg_gene_list = [gene for gene, count in gene_counts.items() if count >= min_sample_occurrence]

print(f"\nTotal number of unique SVGs found in at least {min_sample_occurrence} samples: {len(svg_gene_list)}")

# --- 3. Filter AnnData objects with the final gene list ---
if not svg_gene_list:
    print("\nWarning: No SVGs met the criteria. Consider lowering the `min_sample_occurrence` or p-value threshold.")
else:
    print("Filtering AnnData objects to keep only the identified SVGs...")
    for i, adata in enumerate(adata_rna_list_new):
        # Find the genes that are both in the current adata and our final SVG list
        common_genes = adata.var_names.intersection(svg_gene_list)
        adata_filtered = adata[:, common_genes]
        adata_rna_list[i] = adata_filtered

    print("Filtering complete!")
    if adata_rna_list:
        print("Example shape of the first filtered AnnData object:", adata_rna_list[0].shape)


Finding spatially variable genes (SVGs) present in at least 2 samples...
--- Processing sample: 151673 ---
Found 3770 significant SVGs in sample 151673
--- Processing sample: 151674 ---
Found 3696 significant SVGs in sample 151674
--- Processing sample: 151675 ---
Found 3277 significant SVGs in sample 151675
--- Processing sample: 151676 ---
Found 2857 significant SVGs in sample 151676

Total number of unique SVGs found in at least 2 samples: 3283
Filtering AnnData objects to keep only the identified SVGs...
Filtering complete!
Example shape of the first filtered AnnData object: (3611, 3282)


##### Match

In [22]:
adata_img_list = AnnData.normalize_adata_list(adata_img_list, mode='IMG')

adata: (4992, 3072)
adata: (4992, 3072)
adata: (4992, 3072)
adata: (4992, 3072)
Concatenating IMG data...
All IMG datasets are compatible. Using fast concatenation...


In [23]:
adata_rna_list = AnnData.normalize_adata_list(adata_rna_list, min_genes=1, min_counts=1, mode='RNA',n_top_genes=None)

adata: (3611, 3282)
adata: (3635, 3283)
adata: (3566, 3279)
adata: (3431, 3275)
Starting data concatenation...
Found 3271 common genes
Processing dataset 1/4...
Processing dataset 2/4...
Processing dataset 3/4...
Processing dataset 4/4...
Starting dataset concatenation...
Data concatenation complete!


In [24]:
adata_img_list, adata_rna_list = match_and_filter_adata_lists(adata_img_list, adata_rna_list, match_by='spatial')


--- Processing pair 1 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3611
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3611

--- Processing pair 2 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3635
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3635

--- Processing pair 3 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3566
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3566

--- Processing pair 4 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3431
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3431


In [25]:
true_adata_dict_list = AnnData.generate_MultiOmic_adata_dict_list(Img_list = adata_img_list, RNA_list = adata_rna_list, simulate_missing=False)
adata_cat = AnnData.mode_concat(adata_dict_list=true_adata_dict_list)

['RNA', 'IMG']
Applying special merging rules for 'cell_type'...
  - Number of cell types in adata_cat: 7
  - Number of cell types in adata_m: 1
  -> Decision: Keeping 'cell_type' from adata_cat.
0, IMG
KeysView(AxisArrays with keys: spatial, location, normalized_counts, mask, img_coordinates, IMG_Shape)
Starting to clean DataFrame with 15 columns...
Cleaning complete.
Starting to clean DataFrame with 4 columns...
Cleaning complete.
AnnData object with n_obs × n_vars = 3611 × 6343
    obs: 'IMG_cell_type', 'RNA_cell_type', 'array_col', 'array_row', 'batch', 'batch_view', 'cell_index', 'cell_type', 'in_tissue', 'layers_label', 'mask', 'n_counts', 'n_genes', 'size_factors', 'slice_view'
    var: 'mean', 'std', 'mode', 'mask'
    uns: 'log1p', 'mode', 'slice_id'
    obsm: 'spatial', 'location', 'normalized_counts', 'mask', 'img_coordinates', 'IMG_Shape'
    layers: 'rawX'
Applying special merging rules for 'cell_type'...
  - Number of cell types in adata_cat: 7
  - Number of cell types in

In [ ]:
for n,adata in zip(img_names,adata_cat):
    columns_to_delete = [
    'n_cells-batch_0', 'highly_variable-batch_0', 'n_counts-batch_0',
    'n_cells-batch_1', 'highly_variable-batch_1', 'n_counts-batch_1',
    'n_cells-batch_10', 'highly_variable-batch_10', 'n_counts-batch_10',
    'n_cells-batch_11', 'highly_variable-batch_11', 'n_counts-batch_11',
    'n_cells-batch_2', 'highly_variable-batch_2', 'n_counts-batch_2',
    'n_cells-batch_3', 'highly_variable-batch_3', 'n_counts-batch_3',
    'n_cells-batch_4', 'highly_variable-batch_4', 'n_counts-batch_4',
    'n_cells-batch_5', 'highly_variable-batch_5', 'n_counts-batch_5',
    'n_cells-batch_6', 'highly_variable-batch_6', 'n_counts-batch_6',
    'n_cells-batch_7', 'highly_variable-batch_7', 'n_counts-batch_7',
    'n_cells-batch_8', 'highly_variable-batch_8', 'n_counts-batch_8',
    'n_cells-batch_9', 'highly_variable-batch_9', 'n_counts-batch_9',
    'mean', 'std',
    ]
    adata.var = adata.var.drop(columns=columns_to_delete, errors='ignore')
    adata.write(f'./data/adata_slice{n}.h5ad')

In [26]:
from importlib import reload
import preprocess 
reload(preprocess)
from preprocess import AnnDataProcessor, Config
AnnData = AnnDataProcessor()
adata_img_list = AnnData.create_img_adata_list_from_data(img_list, barcodes_list, img_coordinates_list, xycoords_list, patch_size = 160)

FALSE
Vectorizing and cropping 160x160 image patches from 4992 coordinate points...
AnnData object construction complete!
Vectorizing and cropping 160x160 image patches from 4992 coordinate points...
AnnData object construction complete!
Vectorizing and cropping 160x160 image patches from 4992 coordinate points...
AnnData object construction complete!
Vectorizing and cropping 160x160 image patches from 4992 coordinate points...
AnnData object construction complete!


In [27]:
adata_img_list = AnnData.normalize_adata_list(adata_img_list, mode='IMG')

adata: (4992, 76800)
adata: (4992, 76800)
adata: (4992, 76800)
adata: (4992, 76800)
Concatenating IMG data...
All IMG datasets are compatible. Using fast concatenation...


In [28]:
adata_img_list, adata_rna_list = match_and_filter_adata_lists(adata_img_list, adata_rna_list, match_by='spatial')


--- Processing pair 1 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3611
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3611

--- Processing pair 2 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3635
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3635

--- Processing pair 3 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3566
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3566

--- Processing pair 4 of anndata objects ---
Original sizes: List1 -> 4992, List2 -> 3431
Matching mode: 'spatial'. Synchronizing obs_names with reference list 1...
Size after matching and synchronization: 3431


In [ ]:
for n,adata in zip(img_names, adata_img_list):
    adata.write(f'./data/adata_h_img_{n}.h5ad')